## 📚 Table of Contents

### 1️⃣ Data Loading & Cleaning
- [1.1 📥 Load Dataset](#load-dataset)
- [2.2 🧹 Data Cleaning](#data-cleaning)

### 2️⃣ Distribution Analysis
- [2.1 📊 Distribution of Key Variables](#distribution-variables)
- [2.2 ⚖️ Target Imbalance Check](#target-imbalance)

### 3️⃣ Outlier Removal
- [3.1 📐 Z-score Method](#zscore)
- [3.2 📦 IQR Method](#iqr)

### 4️⃣ Feature Engineering
- [4.1 🛠️ Financial Ratios (Utilization, Payment to Depet Ratio)](#financial-features)

### 5️⃣ RFM Analysis
- [5.1 ⏱️ Recency](#recency)
- [5.2 🔁 Frequency](#frequency)
- [5.3 💰 Monetary](#monetary)

### 6️⃣ EDA
- [6.1 📊 EDA](#eda)

### 7️⃣ Statistical Inference
- [7.1 📏 Confidence Intervals](#confidence-interval)
- [7.2 📊 t-test](#ttest)
- [7.3 🧮 Chi-Square Test](#chi-square)

### 8️⃣ Imbalance Handling
- [8.1 ⚖️ SMOTE Oversampling](#smote)

### 9️⃣ ML Modeling
- [9.1 🤖 Logistic Regression & ROC AUC](#logistic-baseline)
- [9.2 📊 Tree and Gradient Boosting Models](#tree-boosting)
- [9.3 🎯 Best Model](#best-model)

### 🔟 Statistical Sampling
- [10.1 🔄 Bootstrap Sampling](#bootstrap)
- [10.2 🔀 Permutation Testing](#permutation)

### 11 Clustering & Dimensionality Reduction
- [11.1 🌀 KMeans & DBSCAN](#clustering)
- [11.2 📉 PCA Transformation](#pca)


In [ ]:
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

## 1.1 📥 Load Dataset <a id="load-dataset"></a>


In [ ]:
df = pd.read_csv('data/UCI_Credit_Card.csv')
pd.set_option("display.max_columns", None)

df.head()

### There are 25 variables:

- ID: ID of each client
- LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit)
- SEX: Gender (1=male, 2=female)
- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE: Marital status (1=married, 2=single, 3=others)
- AGE: Age in years
- PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
- PAY_2: Repayment status in August, 2005 (scale same as above)
- PAY_3: Repayment status in July, 2005 (scale same as above)
- PAY_4: Repayment status in June, 2005 (scale same as above)
- PAY_5: Repayment status in May, 2005 (scale same as above)
- PAY_6: Repayment status in April, 2005 (scale same as above)
- BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
- BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
- BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
- BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
- BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
- BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
- PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
- PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
- PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
- PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
- PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
- PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
- default.payment.next.month: Default payment (1=yes, 0=no)

## 1.2 🧹 Data Cleaning <a id="data-cleaning"></a>


In [ ]:
df['ID'].nunique()

In [ ]:
df.columns = df.columns.str.lower()

df.rename(columns={'pay_0':'status_sep',
                    'pay_2': 'status_aug',
                    'pay_3': 'status_jul',
                    'pay_4': 'status_june',
                    'pay_5': 'status_may',
                    'pay_6': 'status_apr',
                    'bill_amt1': 'debt_sep',
                    'bill_amt2': 'debt_aug',
                    'bill_amt3': 'debt_jul',
                    'bill_amt4': 'debt_june',
                    'bill_amt5': 'debt_may',
                    'bill_amt6': 'debt_apr',
                    'pay_amt1': 'pay_sep',
                    'pay_amt2': 'pay_aug',
                    'pay_amt3': 'pay_jul',
                    'pay_amt4': 'pay_june',
                    'pay_amt5': 'pay_may',
                    'pay_amt6': 'pay_apr',
                    'default.payment.next.month': 'default_payment'}, inplace=True)

In [ ]:
df.columns

In [ ]:
df.head(10)

In [ ]:
base_cols = [
    "id","limit_bal","sex","education","marriage","age"
]
pay_cols = ['status_sep','status_aug', 'status_jul', 'status_june', 'status_may', 'status_apr']
bill_cols = ['debt_sep','debt_aug', 'debt_jul', 'debt_june', 'debt_may', 'debt_apr']
pay_amt_cols = ['pay_sep','pay_aug', 'pay_jul', 'pay_june', 'pay_may', 'pay_apr']
target_col = ["default_payment"]

df = df[base_cols + pay_cols + bill_cols + pay_amt_cols + target_col]
df.columns

## 2.1 📊 Distribution of Key Variables <a id="distribution-variables"></a>

In [ ]:
df['sex'].value_counts()

In [ ]:
df.hist(figsize=(20, 15), bins=30)
plt.tight_layout()
plt.show()

## 2.2 ⚖️ Target Imbalance Check <a id="target-imbalance"></a>

In [ ]:
df['default_payment'].value_counts()

In [ ]:
df['default_payment'].value_counts().plot.pie(
    autopct='%1.1f%%', 
    figsize=(4, 4), 
    startangle=90
)
plt.ylabel("")
plt.title("Default Payment Distribution")
plt.tight_layout()
plt.show()

#### **the target column is inbalanced so we need to oversample it.**

## 3.1 📐 Z-score Method to remove the outliers <a id="zscore"></a>

### use z-score when the data is normally distributed and IQR when it is not.

In [ ]:
from scipy.stats import zscore
# Example: apply on numeric columns only
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Calculate Z-scores
z_scores = df[numeric_cols].apply(zscore)

# Keep rows where all z-scores are within threshold (e.g., ±3)
df_no_outliers = df[(np.abs(z_scores) < 3).all(axis=1)]

print(f"Original shape: {df.shape}")
print(f"After removing outliers: {df_no_outliers.shape}")

## 3.2 📦 IQR Method to remove the outliers <a id="iqr"></a>

In [ ]:
def remove_outliers_iqr(df):
    """
    Remove outliers from all numeric columns using IQR.
    """
    numeric_cols = df.select_dtypes(include=['number']).columns
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

In [ ]:
df_no_outliers = remove_outliers_iqr(df)
print(f"Original shape: {df.shape}")
print(f"After removing outliers: {df_no_outliers.shape}")

### from this we can conclude that the data contains a lot of outliers, a z-score removed 4,000 rows and IQR removes 22,000 rows because it is more sensitive to the distribution of the data.

## **since the data we use is a banking data and it's ok to have outliers, so we will consider the data without removing outliers and will continue with the original df.**
---


## 4.1 🛠️ Financial Ratios (Utilization, Avg Payment) <a id="financial-features"></a>


Utilization: Shows how much of the credit limit is being used.

In [ ]:
for month in ['apr','may','june','jul','aug','sep']:
    df[f'utilization_{month}'] = df[f'debt_{month}'] / df['limit_bal']
df['avg_utilization'] = df[[f'utilization_{m}' for m in ['apr','may','june','jul','aug','sep']]].mean(axis=1)


Payment-to-Debt Ratio (per month + overall): Measures whether payments are covering outstanding debt. [less than 1, then the payments doesn't cover all the dept]

In [ ]:
for month in ['apr','may','june','jul','aug','sep']:
    df[f'pay_ratio_{month}'] = df[f'pay_{month}'] / (df[f'debt_{month}'] + 1e-6)  # avoid divide by 0
df['avg_pay_ratio'] = df[[f'pay_ratio_{m}' for m in ['apr','may','june','jul','aug','sep']]].mean(axis=1)


pay_ratio_std: Checks stability of payments.

In [ ]:
df['pay_ratio_std'] = df[[f'pay_ratio_{m}' for m in ['apr','may','june','jul','aug','sep']]].std(axis=1)

In [ ]:
df['avg_delay'] = df[[f'status_{m}' for m in ['apr','may','june','jul','aug','sep']]].mean(axis=1)
df['max_delay'] = df[[f'status_{m}' for m in ['apr','may','june','jul','aug','sep']]].max(axis=1)

In [ ]:
dd = df['default_payment']
df.drop('default_payment', axis=1, inplace=True)
df['default_payment'] = dd

In [ ]:
df.columns

In [ ]:
avg_d = df['avg_delay']
max_d = df['max_delay']
df.drop(['avg_delay', 'max_delay'], axis=1, inplace=True)

# Find position after 'status_apr'
idx = df.columns.get_loc('status_apr') + 1

df = pd.concat([df.iloc[:, :idx], avg_d, max_d, df.iloc[:, idx:]], axis=1)

In [ ]:
df.columns

## 5.1 ⏱️ Recency <a id="recency"></a>

In [ ]:
months = ['apr','may','june','jul','aug','sep']
month_order = {m: i+1 for i, m in enumerate(months)}

In [ ]:
# Recency → latest month with nonzero payment
df['R'] = df[[f'pay_{m}' for m in months]].apply(
    lambda row: max([month_order[m] for m in months if row[f'pay_{m}'] > 0], default=0),
    axis=1
)

df['R'].value_counts()

## 5.2 🔁 Frequency <a id="frequency"></a>

In [ ]:
# Frequency → number of trxn made in the period (Apr–Sep) with nonzero payment
df['F'] = df[[f'pay_{m}' for m in months]].gt(0).sum(axis=1)

df['F'].value_counts()

## 5.3 💰 Monetary <a id="monetary"></a>

In [ ]:
# Monetary → total payment across Apr–Sep
df['M'] = df[[f'pay_{m}' for m in months]].sum(axis=1)

In [ ]:
df.head(10)

## 6.1 📊 EDA <a id="eda"></a>


In [ ]:
import math

num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# calculate grid size
n_cols = 4
n_rows = math.ceil(len(num_cols) / n_cols)

plt.figure(figsize=(5*n_cols, 4*n_rows))

for i, column in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.boxplot(x=df[column])
    plt.title(column)

plt.tight_layout()
plt.show()


In [ ]:
corr = df.corr()

# mask for upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(30, 15))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Lower-Triangle Correlation Heatmap")
plt.show()

In [ ]:
# distribution of limit_bil
plt.figure(figsize=(12, 6))
sns.histplot(df['limit_bal'], bins=30, kde=True)
plt.title("Distribution of Credit Limit")
plt.xlabel("Credit Limit")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# relation between Relationship between limit_bal & default rate using line plot
plt.figure(figsize=(20, 5))
sns.lineplot(x='limit_bal', y='default_payment', data=df)
plt.title("Relationship between Credit Limit and Default Rate")
plt.xlabel("Credit Limit")
plt.ylabel("Default Rate")
plt.show()


In [ ]:
# Heatmap of repayment status (status_apr → status_sep).
repayment_status = df.filter(like='status_')
plt.figure(figsize=(12, 8))
mask = np.triu(np.ones_like(repayment_status.corr(), dtype=bool))
sns.heatmap(repayment_status.corr(), mask=mask, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Heatmap of Repayment Status")
plt.show()

In [ ]:
# Trend lines: average debt and payment over months
debt_cols = ['debt_apr', 'debt_may', 'debt_june', 'debt_jul', 'debt_aug', 'debt_sep']
pay_cols  = ['pay_apr',  'pay_may',  'pay_june',  'pay_jul',  'pay_aug',  'pay_sep']

# Calculate averages for each month
avg_debt = df[debt_cols].mean()
avg_pay  = df[pay_cols].mean()

# Create dataframe for plotting
trend_df = pd.DataFrame({
    "Month": ["Apr", "May", "Jun", "Jul", "Aug", "Sep"],
    "Avg Debt": avg_debt.values,
    "Avg Payment": avg_pay.values
})

# Plot
plt.figure(figsize=(10,6))
plt.plot(trend_df["Month"], trend_df["Avg Debt"], marker='o', label="Average Debt")
plt.plot(trend_df["Month"], trend_df["Avg Payment"], marker='o', label="Average Payment")
plt.title("Trend of Average Debt and Payment (Apr - Sep)")
plt.xlabel("Month")
plt.ylabel("Amount")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Target correlation plot (which features are most predictive of default_payment).
target_corr = df.corr()['default_payment'].sort_values(ascending=False)
plt.figure(figsize=(20, 8))
sns.barplot(x=target_corr.index, y=target_corr.values)
plt.title("Feature Correlation with Default Payment")
plt.xticks(rotation=45)
plt.show()

## 7.1 📏 Confidence Intervals<a id="confidence-interval"></a>
### get the confidence interval of a mean (or any statistic) that the population is likely to fall into
CI = x` ± Z(α/2) * (​σ​​/sqrt(n))

In [ ]:
import scipy.stats as st

mean = df['limit_bal'].mean()
n = len(df)
std = np.std(df['limit_bal'], ddof=1)
alpha = 0.05
z = st.norm.ppf(1 - alpha/2)
margin_of_error = z * (std / np.sqrt(n))
ci_lower, ci_upper = mean - margin_of_error, mean + margin_of_error
print(f"95% Confidence Interval for limit_bal: [{ci_lower:.2f}, {ci_upper:.2f}]")

In [ ]:
ci = st.t.interval(
    confidence=0.95,  # CI
    df=len(df['limit_bal'])-1,  # margin error
    loc=np.mean(df['limit_bal']), 
    scale=st.sem(df['limit_bal'])  # standard error of mean
)

print(f"95% CI (using t-distribution): {ci}")

## 7.2 📊 t-test<a id="ttest"></a>

### t-test: compare the value of mean (any statistic) of the default payment group vs non-default payment group to see if there is a statistically significant difference between the two groups.
### if p-value < α (0.05) then we reject the null hypothesis (the means are equal).

In [ ]:
import scipy.stats as stats

# One-sample t-test
stats.ttest_1samp(df['age'], popmean=35)

# Independent two-sample t-test (e.g., limit_bal between default vs non-default)
group1 = df[df['default_payment'] == 0]['limit_bal']
group2 = df[df['default_payment'] == 1]['limit_bal']

print('Group 1 (have not defaulted):',group1.mean())
print('Group 2 (have defaulted):',group2.mean())
print(stats.ttest_ind(group1, group2))


**The p-value is very less than the significance level of 0.05, so We reject the null hypothesis (the means are equal).**

**We can conclude that there is a statistically significant difference between the means of the two groups.**

## 7.3 🧮 Chi-Square Test<a id="chi-square"></a>
### Chi-Square Test: used to determine if there is a significant association between two categorical variables.
### example: is there is a relationship between default payment and marital status?

In [ ]:
df.columns

In [ ]:
from scipy.stats import chi2_contingency

categorical_features = ["sex", "marriage", "education", "avg_delay"]  

results = {}

for col in categorical_features:
    contingency_table = pd.crosstab(df[col], df["default_payment"])  # observed table
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    results[col] = {
        "chi2_statistic": chi2,
        "p_value": p,
        "degrees_of_freedom": dof,
        "significant": p < 0.05
    }

results_df = pd.DataFrame(results).T
print(results_df)

**there is significant relationship between education and avg_delay with default_payment (the highest two chi2 values)**

## 8.1 ⚖️ SMOTE Oversampling<a id="smote"></a>
since the target column is imbalanced, we can use SMOTE to generate synthetic samples for the minority class.

In [ ]:
df['default_payment'].value_counts()

In [ ]:
X = df.drop(columns=["default_payment"])
y = df["default_payment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", Counter(y_train))
print("After SMOTE:", Counter(y_resampled))

In [ ]:
# scaling using RobustScaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_resampled = scaler.fit_transform(X_resampled)
X_test = scaler.transform(X_test)


In [ ]:
# save the scaler at models directory
import joblib
joblib.dump(scaler, 'models/robust_scaler.pkl')

## 9.1 🤖 Logistic Regression & ROC AUC<a id="logistic-baseline"></a>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

model = LogisticRegression(max_iter=1000)
model.fit(X_resampled, y_resampled)

In [ ]:
y_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = roc_auc_score(y_test, y_probs)

J = tpr - fpr
best_idx = np.argmax(J)
best_threshold = thresholds[best_idx]

print("AUC:", roc_auc)
print("Best threshold:", best_threshold)

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0,1], [0,1], 'k--')
plt.scatter(fpr[best_idx], tpr[best_idx], marker='o', color='red', label=f'Best Threshold = {best_threshold:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

y_pred_best = (y_probs >= best_threshold).astype(int)

cm = confusion_matrix(y_test, y_pred_best)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n", classification_report(y_test, y_pred_best))

print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Precision:", precision_score(y_test, y_pred_best))
print("Recall:", recall_score(y_test, y_pred_best))
print("F1 Score:", f1_score(y_test, y_pred_best))

## 9.2📊 Tree and Gradient Boosting Models<a id="tree-boosting"></a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_resampled, y_resampled)

In [ ]:
y_probs_rf = rf_model.predict_proba(X_test)[:, 1]
best_threshold_rf = 0.5 
y_pred_rf = (y_probs_rf >= best_threshold_rf).astype(int)

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:\n", cm_rf)

print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1 Score:", f1_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_probs_rf))

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

In [ ]:
y_prob_xg = xgb_model.predict_proba(X_test)[:, 1]
best_threshold_xg = 0.5
y_pred_xg= (y_probs_rf >= best_threshold_xg).astype(int)

In [ ]:
cm_xg= confusion_matrix(y_test, y_pred_xg)
print("Confusion Matrix:\n", cm_xg)

print("\nClassification Report:\n", classification_report(y_test, y_pred_xg))

print("Accuracy:", accuracy_score(y_test, y_pred_xg))
print("Precision:", precision_score(y_test, y_pred_xg))
print("Recall:", recall_score(y_test, y_pred_xg))
print("F1 Score:", f1_score(y_test, y_pred_xg))
print("ROC AUC:", roc_auc_score(y_test, y_prob_xg))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm_xg = confusion_matrix(y_test, y_pred_xg)
disp_xg = ConfusionMatrixDisplay(confusion_matrix=cm_xg, display_labels=xgb_model.classes_)
disp_xg.plot(cmap='Blues')

## 9.3 🎯 Best Model<a id="best-model"></a>

In [ ]:
import joblib
joblib.dump(xgb_model, 'models/xgboost.pkl')

## 10.1 🔄 Bootstrap Sampling<a id="bootstrap"></a>

### bootstrap sampling to see the CI for each metric to the model if it fitted with a population data

In [ ]:
from sklearn.utils import resample, shuffle
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
n_bootstrap = 1000
bootstrap_results = {m: [] for m in metrics}

for i in range(n_bootstrap):
    indices = np.random.choice(len(y_test), len(y_test), replace=True)
    y_true_bs = y_test.iloc[indices]
    y_pred_bs = y_pred_xg[indices]
    y_prob_bs = y_prob_xg[indices]
    
    bootstrap_results['accuracy'].append(accuracy_score(y_true_bs, y_pred_bs))
    bootstrap_results['precision'].append(precision_score(y_true_bs, y_pred_bs))
    bootstrap_results['recall'].append(recall_score(y_true_bs, y_pred_bs))
    bootstrap_results['f1'].append(f1_score(y_true_bs, y_pred_bs))
    bootstrap_results['roc_auc'].append(roc_auc_score(y_true_bs, y_prob_bs))

# حساب 95% Confidence Interval
for m in metrics:
    lower = np.percentile(bootstrap_results[m], 2.5)
    upper = np.percentile(bootstrap_results[m], 97.5)
    print(f"95% CI for {m}: [{lower:.4f}, {upper:.4f}]")

## 10.2 🔀 Permutation Testing<a id="permutation"></a>
### Permutation Test is to shuffle the labels and see if the model can still predict well

In [ ]:
observed_auc = roc_auc_score(y_test, y_prob_xg)
n_permutations = 1000
count = 0

for _ in range(n_permutations):
    y_permuted = shuffle(y_test)
    perm_auc = roc_auc_score(y_permuted, y_prob_xg)
    if perm_auc >= observed_auc:
        count += 1

p_value = count / n_permutations
print(f"Permutation test p-value for ROC AUC: {p_value:.4f}")

**the model is not a coincidence and the prediction result is real since p-value > 0.05**

## 11.1 🌀 KMeans & DBSCAN<a id="clustering"></a>

 **first choice of us is to drop the outliers using IQR method since our data is not normally distributed, and then use K-mean clustering (this step is important because K mean is senstive to outliers)**

In [ ]:
features = ['limit_bal', 'avg_delay', 'avg_utilization', 'avg_pay_ratio', 'R', 'F', 'M']
X = df[features]
X = remove_outliers_iqr(X)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

inertia = []
K_range = range(2, 11)  
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8,4))
plt.plot(K_range, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
silhouette_scores = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(8,4))
plt.plot(K_range, silhouette_scores, 'ro-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis')
plt.show()

In [ ]:
optimal_k = 3
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42)
X['cluster'] = kmeans_final.fit_predict(X)

print(X['cluster'].value_counts())

---
**second choice is to continue with the original data without removing outliers and use DBSCAN because it is less sensitive to outliers.**

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import RobustScaler, StandardScaler

features = [
    'limit_bal', 'sex', 'education', 'marriage', 'age', 
    'status_sep', 'status_aug', 'status_jul', 'status_june', 'status_may', 'status_apr',
    'avg_delay', 'max_delay', 'debt_sep', 'debt_aug', 'debt_jul', 'debt_june', 'debt_may', 'debt_apr',
    'pay_sep', 'pay_aug', 'pay_jul', 'pay_june', 'pay_may', 'pay_apr',
    'utilization_apr', 'utilization_may', 'utilization_june', 'utilization_jul', 'utilization_aug', 'utilization_sep',
    'avg_utilization', 'pay_ratio_apr', 'pay_ratio_may', 'pay_ratio_june', 'pay_ratio_jul', 'pay_ratio_aug', 'pay_ratio_sep',
    'avg_pay_ratio', 'pay_ratio_std', 'R', 'F', 'M'
]

X = df[features].values
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)



## 11.2 📉 PCA Transformation<a id="pca"></a>

In [ ]:
pca = PCA()
pca.fit(X_scaled)

explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = explained_variance_ratio.cumsum()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(1, len(cumulative_variance)+1), cumulative_variance, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs Number of Components')
plt.grid(True)
plt.show()


In [ ]:
n_components_optimal = (cumulative_variance >= 0.95).argmax() + 1
print("Optimal number of components:", n_components_optimal)

pca_optimal = PCA(n_components=n_components_optimal)
X_pca = pca_optimal.fit_transform(X_scaled)

In [ ]:
pca = PCA(n_components=5, random_state=42)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
dbscan = DBSCAN(eps=1.5, min_samples=500) 
clusters = dbscan.fit_predict(X_pca)

print("Cluster distribution:", Counter(clusters))